### Extract file names from dataset

In [1]:
import os

dataset_dir = "/mnt/linux/abrc/abrc/dataset/stomaVDP"
dataset_name = "label-Leafnet-and-others"

img_extention = ["jpg", "png", "tif"]
parse_dirs = ["train", "val"]

for d in parse_dirs:
    img_list = os.listdir(os.path.join(dataset_dir, dataset_name, "images", d))
    output_filename = os.path.join(dataset_dir, dataset_name, d + ".txt")

    # clear data in the output_file
    with open(output_filename, "w") as f:
        pass

    for idx, img in enumerate(img_list):
        if img.split(".")[-1] in img_extention:
            print(img)
            with open(output_filename, "a") as f:
                f.write(os.path.join("./images", d, img))
                if idx < len(img_list) - 1:
                    f.write("\n")

print("file name extraction complete......")

BE_20X013.jpg
10441.jpg
p_tr_38.png
10090.jpg
c_tr_87_cut_want.jpg
u_tr_184.png
u_tr_26.png
107.jpg
p_tr_63.png
u_tr_42.png
c_tr_99_cut_want.jpg
u_tr_74.png
u_tr_59.png
p_tr_68.png
10060.jpg
9999.jpg
u_tr_227.png
C1L004.jpg
c_tr_154_cut_want.jpg
c_tr_13_cut_want.jpg
u_tr_283.png
31.jpg
c_tr_159_cut_want.jpg
u_tr_54.png
u_tr_46.png
p_tr_57.png
c_tr_62_cut_want.jpg
u_tr_118.png
u_tr_28.png
C1L003.jpg
c_tr_105_cut_want.jpg
56.jpg
u_tr_39.png
1_c_tr_6_cut_want.jpg
u_tr_72.png
C1L008.jpg
BE_20X012.jpg
p_tr_91.png
c_tr_65_cut_want.jpg
BE_20X009.jpg
CK001.jpg
u_tr_12.png
90.jpg
p_tr_66.png
p_tr_4.png
u_tr_222.png
12.jpg
c_tr_24_cut_want.jpg
u_tr_293.png
C1L009.jpg
p_tr_52.png
u_tr_95.png
c_tr_30_cut_want.jpg
c_tr_124_cut_want.jpg
12782.jpg
p_tr_33.png
c_tr_176_cut_want.jpg
u_tr_7.png
p_tr_83.png
u_tr_290.png
15.jpg
p_tr_98.png
186.jpg
10203.jpg
p_tr_58.png
1.jpg
51.jpg
u_tr_238.png
c_tr_70_cut_want.jpg
c_tr_56_cut_want.jpg
p_tr_74.png
10221.jpg
p_tr_43.png
p_tr_116.png
u_tr_84.png
10473.jpg
u

### Convert Labels from COCO JSON instance to Yolov7 compatible polygon label format

In [2]:
import os, json, shutil

dataset_dir = "/mnt/linux/abrc/abrc/dataset/stomaVDP"
dataset_name = "label-Leafnet-and-others"

img_extention = ["jpg", "png", "tif"]
parse_dirs = ["train", "val"]

for d in parse_dirs:
    print(f"Start extracting data in {d}........")
    # Load label JSON file
    orig_annotations = None
    with open(
        os.path.join(dataset_dir, dataset_name, "labels", f"labels_{d}.json"), "r"
    ) as f_input:
        orig_annotations = json.load(f_input)

    dir = os.path.join(dataset_dir, dataset_name, "labels", d)
    CHECK_FOLDER = os.path.isdir(dir)

    if not CHECK_FOLDER:  # If folder doesn't exist, then create it.
        os.makedirs(os.path.join(dir, "seg"))
        os.makedirs(os.path.join(dir, "det"))
        print("created folder : ", dir)
    else:  # Reset folder if it exists.
        print(dir, "folder already exists.")
        shutil.rmtree(dir, ignore_errors=True)
        print("delete folder : ", dir)
        os.makedirs(os.path.join(dir, "seg"))
        os.makedirs(os.path.join(dir, "det"))
        print("created folder : ", dir)

    # Iterate through images in the JSON file
    for i, img in enumerate(orig_annotations):
        # Save Yolov7 annotation for instance segmentation
        output_filename = img["file_name"].split("/")[-1]  # get image name
        output_filename = os.path.splitext(output_filename)[0]  # remove extension
        output_filename = os.path.join(
            dataset_dir, dataset_name, "labels", d, f"{output_filename}.txt"
        )  # get full file name

        print(
            f"[{i+1}/{len(orig_annotations)}] extract and load seg annotations to: {output_filename}"
        )

        x_max = img["width"]
        y_max = img["height"]

        # Prepare and save segmentation annotations
        with open(output_filename, "w") as f_output:
            # iterate through annotations in each image.
            for idx, anno in enumerate(img["annotations"]):
                output = ""  # reset output
                output = output + str(anno["category_id"]) + " "  # add category id
                # save segmentation
                for idy, num in enumerate(anno["segmentation"][0]):
                    # normalise annotation
                    if idy % 2 == 0:  # odd number
                        output = output + str(round(num / x_max, 6))
                        if num / x_max > 1:
                            print(f"X wrong!!!: x={num}, xmax={x_max}")
                    else:  # even number
                        output = output + str(round(num / y_max, 6))
                        if num / y_max > 1:
                            print(f"Y wrong!!!: y={num}, ymax={y_max}")
                    if idy < len(anno["segmentation"][0]) - 1:
                        output = output + " "
                if idx < len(img["annotations"]) - 1:
                    output = output + "\n"
                f_output.write(output)

        # Save Yolov7 annotation for object detection
        output_filename = img["file_name"].split("/")[-1]  # get image name
        output_filename = os.path.splitext(output_filename)[0]  # remove extension
        output_filename = os.path.join(
            dataset_dir, dataset_name, "labels", d, "det", f"{output_filename}.txt"
        )  # get full file name

        print(
            f"[{i+1}/{len(orig_annotations)}] extract and load det annotations to: {output_filename}"
        )

        x_max = img["width"]
        y_max = img["height"]

        # Prepare and save detection annotations
        with open(output_filename, "w") as f_output:
            # iterate through annotations in each image.
            for idx, anno in enumerate(img["annotations"]):
                # reset output
                output = ""

                """
                Bbox annotation in Detectron2 is:
                anno['bbox'][0]: left top x
                anno['bbox'][1]: left top y
                anno['bbox'][1]: right bottom x
                anno['bbox'][1]: right bottom y
                """
                x_0 = anno["bbox"][0]
                y_0 = anno["bbox"][1]
                x_1 = anno["bbox"][2]
                y_1 = anno["bbox"][3]

                # add category id
                output = output + str(anno["category_id"]) + " "

                # add centre x axis
                x_centre = round((x_0 + x_1) / (2 * x_max), 6)
                if x_centre > 1:
                    print(f"X centre is wrong!!! centre_x={x_centre}, xmax={x_max}")
                output = output + str(x_centre) + " "

                # add centre y axis
                y_centre = round((y_0 + y_1) / (2 * y_max), 6)
                if y_centre > 1:
                    print(f"Y centre is wrong!!!: centre_x={y_centre}, ymax={y_max}")
                output = output + str(y_centre) + " "

                # add width
                width = round((x_1 - x_0) / (x_max), 6)
                if (width / 2) + x_centre > x_max:
                    print(f"width is wrong: width={width}, xmax={x_max}")
                if width < 0:
                    print(f"width is minus: width={width}, xmax={x_max}")
                output = output + str(width) + " "

                # add height
                height = round((y_1 - y_0) / (y_max), 6)
                if (height / 2) + y_centre > y_max:
                    print(f"width is wrong: height={height}, xmax={y_max}")
                if height < 0:
                    print(f"height is minus: width={height}, xmax={y_max}")
                output = output + str(height)

                if idx < len(img["annotations"]) - 1:
                    output = output + "\n"
                f_output.write(output)

Start extracting data in train........
created folder :  /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train
[1/188] extract and load seg annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/c_tr_97_cut_want.txt
[1/188] extract and load det annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/det/c_tr_97_cut_want.txt
[2/188] extract and load seg annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/10294.txt
[2/188] extract and load det annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/det/10294.txt
[3/188] extract and load seg annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/10425.txt
[3/188] extract and load det annotations to: /mnt/linux/abrc/abrc/dataset/stomaVDP/label-Leafnet-and-others/labels/train/det/10425.txt
[4/188] extract and load seg annotations to: /m